# AIRLINE - CUSTOMER SATISFACTION

![Customer satisfaction](Daco_4436699.png)

### <span style="color:gray">**ÍNDICE**</span>

1) Abstract
2) Data Wrangling

### <span style="color:gray">**1. ABSTRACT**</span>

La premisa inicial consiste en la creación de un modelo destinado a anticipar las decisiones de compra de pasajeros de buses de larga distancia, basándose en su historial de comportamiento o del comportamiento de usuarios con atributos similares. Dada la ausencia de un conjunto de datos específico para este propósito, se utilizó información proveniente de una aerolínea como sustituto temporal, con la posibilidad de replicar el proceso en el eventual acceso a la información del servicio mencionado.

### <span style="color:gray">**2. DATA WRANGLING**</span>

#### <span style="color:gray">**2.1. Antes de empezar**</span>

In [1]:
import pandas as pd
import numpy as np
from colorama import Fore, Style

import warnings

# Desactivar advertencias específicas
warnings.filterwarnings("ignore", category=UserWarning, module="seaborn")

colores = ['#0033a0', '#eb0029'] #azul y rojo
colores_satif = ['#eb0029', '#8DB600'] #rojo y verde manzana

def format_thousands(x, pos):
    return '{:,.0f}'.format(x).replace(',', '.')

#### <span style="color:gray">**2.2. Descubrimiento**</span>
Antes de empezar cualquier análisis, es importante comprender los datos, la estructura, tipos y cantidad.

In [41]:
# Importo el dataset
print(Style.BRIGHT+Fore.LIGHTMAGENTA_EX+'¿Qué información tiene el dataset?'+Style.RESET_ALL)
import pandas as pd
df_airline = pd.read_csv("C:\\Users\\Flori\\CODERHOUSE\\airlinesatisfaction.csv")  
df = df_airline.sort_values(by='satisfaction', ascending=False)

# Información de cada campo
print(df.info())

print()

# Información de cada campo object
print(Style.BRIGHT+Fore.LIGHTMAGENTA_EX+'Datos en cada campo object'+Style.RESET_ALL)
satisf = df['satisfaction'].unique()
cust_type = df['Customer Type'].unique()
type_travel = df['Type of Travel'].unique()
cust_class = df['Class'].unique()
print(satisf)
print(cust_type)
print(type_travel)
print(cust_class)

# Cambio el formato de los campos objet a mayúscula
df['satisfaction'] = df['satisfaction'].str.title()
df['Customer Type'] = df['Customer Type'].str.title()
df['Type of Travel'] = df['Type of Travel'].str.title()
df['Class'] = df['Class'].str.title()

print()

# Traigo las primeras filas para ver cómo queda el dataset con los cambios realizados
print(Style.BRIGHT+Fore.LIGHTMAGENTA_EX+'Preview del dataset con los cambios'+Style.RESET_ALL)
df.head()

¿Qué información tiene el dataset?
<class 'pandas.core.frame.DataFrame'>
Index: 129880 entries, 0 to 129879
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   satisfaction                       129880 non-null  object 
 1   Gender                             129880 non-null  object 
 2   Customer Type                      129880 non-null  object 
 3   Age                                129880 non-null  int64  
 4   Type of Travel                     129880 non-null  object 
 5   Class                              129880 non-null  object 
 6   Flight Distance                    129880 non-null  int64  
 7   Seat comfort                       129880 non-null  int64  
 8   Departure/Arrival time convenient  129880 non-null  int64  
 9   Food and drink                     129880 non-null  int64  
 10  Gate location                      129880 non-null  int64  
 11  Inflight 

,satisfaction,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,Satisfied,Female,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,...,2,3,3,0,3,5,3,2,0,0.0
93280,Satisfied,Male,Loyal Customer,58,Business Travel,Business,1614,5,5,5,...,4,4,4,4,4,5,4,3,0,0.0
93290,Satisfied,Male,Loyal Customer,47,Business Travel,Business,1594,5,5,5,...,4,4,4,4,4,3,4,3,0,0.0
93289,Satisfied,Female,Loyal Customer,52,Business Travel,Business,1578,5,5,5,...,4,4,4,4,4,4,4,5,0,1.0
93288,Satisfied,Male,Loyal Customer,45,Business Travel,Business,1579,4,4,4,...,4,4,4,4,4,5,4,4,0,0.0


#### <span style="color:gray">**2.3. Limpieza**</span>
Debemos eliminar los datos que no brinden información extra como los duplicados, revisar datos faltantes, etc. Esta propiedad estandariza el formato de las columnas (float, datatimes, etc).

###### <span style="color:gray">**2.3.1. Duplicados**</span>

In [16]:
# Busco duplicados
print(Style.BRIGHT+Fore.LIGHTMAGENTA_EX+'Duplicados'+Style.RESET_ALL)

if df.duplicated().any() == False:
    print('No existen filas duplicadas, por lo que no es necesario eliminar registros.')
else:
    print('Existen duplicados. Si desea eliminarlos, usar la función: df = df.drop_duplicates()')


Duplicados
No existen filas duplicadas, por lo que no es necesario eliminar registros.


###### <span style="color:gray">**2.3.2. Datos faltantes**</span>

In [30]:
# Busco datos faltantes
print(Style.BRIGHT+Fore.LIGHTMAGENTA_EX+'Datos faltantes'+Style.RESET_ALL)

nulos = df.isnull().sum()
nulos_pct = nulos / df.shape[0]*100
nulos_resumen = pd.DataFrame({'Cant_Nulos': nulos, 'Pct_Nulos [%]': nulos_pct})
nulos_resumen = nulos_resumen.loc[nulos_resumen['Cant_Nulos'] > 0, :]
nulos_resumen = nulos_resumen.sort_values(by='Cant_Nulos', ascending=False)

nulos_resumen

Datos faltantes


,Cant_Nulos,Pct_Nulos [%]
Arrival Delay in Minutes,393,0.302587


In [40]:
# Reemplazo los datos faltantes por la mediana por la naturaleza de los datos
print(Style.BRIGHT+Fore.LIGHTMAGENTA_EX+'Reemplazo los datos faltantes'+Style.RESET_ALL+' - Reemplazo por cero que es la mediana de los datos por la naturaleza de los datos, minutos de retraso en la llegada')
print()
print('La mediana de la columna Arrival Delay in Minutes es: '+str(df['Arrival Delay in Minutes'].median())+' minutes')

# Reemplazo los datos faltantes por cero que es la mediana
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].mean())

# Verifico que no haya más nulos
nulos = df.isnull().sum()
nulos[nulos>0]

Reemplazo los datos faltantes - Reemplazo por cero que es la mediana de los datos por la naturaleza de los datos, minutos de retraso en la llegada

La mediana de la columna Arrival Delay in Minutes es: 0.0 minutes


Series([], dtype: int64)

#### <span style="color:gray">**2.4. Enriquecimiento**</span>
Esta etapa se refiere a agregar datos extra que complementan a los que ya existen para agregar información extra al análisis.

In [43]:
# Creo una columna satisfaccion binaria que indica 1 si la respuesta es satisfied y 0 en caso contrario para poder manipular mas facil los datos
satisf_binaria = df['satisfaction'].apply(lambda x: 1 if x == 'Satisfied' else 0)
df['satisf_binaria'] = satisf_binaria

df.info()

df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 129880 entries, 0 to 129879
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   satisfaction                       129880 non-null  object 
 1   Gender                             129880 non-null  object 
 2   Customer Type                      129880 non-null  object 
 3   Age                                129880 non-null  int64  
 4   Type of Travel                     129880 non-null  object 
 5   Class                              129880 non-null  object 
 6   Flight Distance                    129880 non-null  int64  
 7   Seat comfort                       129880 non-null  int64  
 8   Departure/Arrival time convenient  129880 non-null  int64  
 9   Food and drink                     129880 non-null  int64  
 10  Gate location                      129880 non-null  int64  
 11  Inflight wifi service              129880 no

,satisfaction,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,...,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,satisf_binaria
0,Satisfied,Female,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,...,3,3,0,3,5,3,2,0,0.0,1
93280,Satisfied,Male,Loyal Customer,58,Business Travel,Business,1614,5,5,5,...,4,4,4,4,5,4,3,0,0.0,1
93290,Satisfied,Male,Loyal Customer,47,Business Travel,Business,1594,5,5,5,...,4,4,4,4,3,4,3,0,0.0,1
93289,Satisfied,Female,Loyal Customer,52,Business Travel,Business,1578,5,5,5,...,4,4,4,4,4,4,5,0,1.0,1
93288,Satisfied,Male,Loyal Customer,45,Business Travel,Business,1579,4,4,4,...,4,4,4,4,5,4,4,0,0.0,1
